THE FILE IS IMPLEMENTATION OF RIGID FRAMEWORK   
**paper link**: https://arxiv.org/abs/2405.20112

In [ ]:
import os
from PIL import Image
from sklearn.metrics import precision_score, recall_score, f1_score

import torch
import torch.nn.functional as F
import torchvision.models as models
import torchvision.transforms as transforms

**LOADING MODEL**


In [ ]:
# Load model directly
from transformers import AutoImageProcessor, AutoModel

processor = AutoImageProcessor.from_pretrained("facebook/dinov2-base")
model = AutoModel.from_pretrained("facebook/dinov2-base")

model.eval().to("cuda")

In [ ]:
def load_image(image_path):
    image = Image.open(image_path).convert("RGB")
    transform = transforms.Compose(
        [
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010]
            ),
        ]
    )
    image = transform(image)
    return image.unsqueeze(0).to("cuda")

In [ ]:
def add_noise(image, noise_level=0.05):
    noise = torch.randn(image.size(), device=image.device) * noise_level

    return image + noise


def add_blur(image, sigma=1.0):
    transform = transforms.GaussianBlur(kernel_size=5, sigma=sigma)
    image_pil = transforms.ToPILImage()(image.cpu())
    blurred_image = transform(image_pil)

    return transforms.ToTensor()(blurred_image).to("cuda")

In [ ]:
def get_feature_dino(model, image):
    with torch.no_grad():
        image_features = model(image).last_hidden_state

    return image_features.mean(dim=1)


def get_feature_resnet(model, image):
    with torch.no_grad():
        image_features = model(image)

    return image_features

In [ ]:
def cosine_sim(feat1, feat2):
    return F.cosine_similarity(feat1, feat2).item()

In [ ]:
def is_ai_resnet(image_path, model, threshold=0.95, noise_level=0.05):
    original = load_image(image_path)
    perturbed_image = add_noise(original, noise_level=noise_level)

    original_feature = get_feature_resnet(model, original)
    perturbed_feature = get_feature_resnet(model, perturbed_image)
    similarity = cosine_sim(original_feature, perturbed_feature)

    return similarity < threshold


def is_ai_dino(image_path, model, threshold=0.95, noise_level=0.05):
    original = load_image(image_path)
    perturbed_image = add_noise(original, noise_level=noise_level)

    original_feature = get_feature_dino(model, original)
    perturbed_feature = get_feature_dino(model, perturbed_image)
    similarity = cosine_sim(original_feature, perturbed_feature)

    return similarity < threshold

In [ ]:
test_folder = "/content/test"
real = os.path.join(test_folder, "REAL")
ai = os.path.join(test_folder, "FAKE")

In [ ]:
true_labels = []
predicted_labels = []
image_count = 0

In [ ]:
def classify_folder_dino(folder, label, model, threshold=0.95):
    global image_count
    for filename in os.listdir(folder):
        image_path = os.path.join(folder, filename)
        if os.path.isfile(image_path):
            is_ai_image = is_ai_dino(image_path, model, threshold)
            true_labels.append(label)
            predicted_labels.append(1 if is_ai_image else 0)
            image_count += 1
            if image_count % 100 == 0:
                print(f"Processed {image_count} images")

In [ ]:
def classify_folder_resnet(folder, label, model, threshold=0.95):
    global image_count
    for filename in os.listdir(folder):
        image_path = os.path.join(folder, filename)
        if os.path.isfile(image_path):
            is_ai_image = is_ai_resnet(image_path, model, threshold)
            true_labels.append(label)
            predicted_labels.append(1 if is_ai_image else 0)
            image_count += 1
            if image_count % 100 == 0:
                print(f"Processed {image_count} images")

In [ ]:
def tune_threshold_resnet(real_folder, fake_folder, model, noise_level=0.05):
    best_threshold = 0.95
    best_f1 = 0
    thresholds = [0.80, 0.85, 0.90, 0.92, 0.95, 0.97]

    for threshold in thresholds:
        global true_labels
        global predicted_labels
        true_labels = []
        predicted_labels = []

        classify_folder_resnet(real_folder, 0, model, threshold)
        classify_folder_resnet(fake_folder, 1, model, threshold)

        precision = precision_score(true_labels, predicted_labels)
        recall = recall_score(true_labels, predicted_labels)
        f1 = f1_score(true_labels, predicted_labels)

        print(f"Threshold: {threshold}")
        print(f"Precision: {precision}")
        print(f"Recall: {recall}")
        print(f"F1 Score: {f1}")

        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold

    return best_threshold

In [ ]:
model2 = models.resnet18(pretrained=True)
model2 = model2.eval().to("cuda")

best_threshold = tune_threshold_resnet(real, ai, model2, noise_level=0.05)

print(best_threshold)

0.97


In [ ]:
def tune_threshold_resnet(real_folder, fake_folder, model, noise_level=0.05):
    best_threshold = 0.95
    best_f1 = 0
    thresholds = [0.95, 0.96, 0.97, 0.98, 0.99, 0.995, 0.999]
    for threshold in thresholds:
        global true_labels
        global predicted_labels
        true_labels = []
        predicted_labels = []

        classify_folder_resnet(real_folder, 0, model, threshold)
        classify_folder_resnet(fake_folder, 1, model, threshold)

        precision = precision_score(true_labels, predicted_labels)
        recall = recall_score(true_labels, predicted_labels)
        f1 = f1_score(true_labels, predicted_labels)

        print(f"Threshold: {threshold}")
        print(f"Precision: {precision}")
        print(f"Recall: {recall}")
        print(f"F1 Score: {f1}")

        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold

    return best_threshold

In [ ]:
best_threshold = tune_threshold_resnet(real, ai, model2, noise_level=0.05)